In [1]:

import numpy as np
import pandas as pd
import datetime
import glob
#from tqdm.auto import tqdm
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from collections import deque

#from dash import Dash, html, dcc
import tkinter
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

import warnings
warnings.filterwarnings("ignore")


In [2]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
report_path = config['base']['report_path']
report_plot_path = config['base']['report_plot_path']
trading_day_date_path = config['base']['trading_day_date_path']
file_type = config['base']['file_type']
img_path = config['base']['img_path']
twa02_path = config['base']['twa02_path']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)
intraday_start_day = datetime.strptime(config['base']['intraday_start_day'], '%Y-%m-%d')
year_days = int(config['base']['year_days'])
transaction_cost = float(config['base']['transaction_cost'])
risk_free_rate = float(config['base']['risk_free_rate'])

total_asset = float(config['allocation']['total_asset'])
risk_factor = float(config['allocation']['risk_factor'])
strategy_risk_factor = float(config['allocation']['strategy_risk_factor'])
stock_risk_factor = float(config['allocation']['stock_risk_factor'])

portfolio_intraday_init_asset = float(config['portfolio_threshold']['portfolio_intraday_init_asset'])

backtesting_start_date = datetime(2021,1,1)


In [3]:

from backtesting.strategy import Strategy
from backtesting.portfolio import Portfolio_intraday, Portfolio_interday, Portfolio
from backtesting.evaluate import get_MDD
from backtesting.evaluate import evaluate_portfolio

portfolio_interday_paths = glob.glob('./portfolio_interday/' + '*' + file_type)
portfolio_intraday_paths = glob.glob('./portfolio_intraday/' + '*' + file_type)

portfolio_cols = []
if portfolio_intraday_paths:
    print('\r Preparing intraday ...', end = '', flush=True)
    portfolio_intraday = Portfolio_intraday(name = 'portfolio_intraday', portfolio_intraday_init_asset = portfolio_intraday_init_asset, portfolio_path = portfolio_intraday_paths)
    #portfolio_intraday.get_plotly_html()
    portfolio_cols += portfolio_intraday.portfolio_cols
    portfolio_df = portfolio_intraday.daily_report
    portfolio_trading_record_df = portfolio_intraday.tradingRecords

if portfolio_interday_paths:
    print('\r Preparing interday ...', end = '', flush=True)
    portfolio_interday = Portfolio_interday(name = 'portfolio_interday', portfolio_path = portfolio_interday_paths)
    #portfolio_interday.get_plotly_html()
    portfolio_cols += portfolio_interday.portfolio_cols
    portfolio_df = portfolio_interday.daily_report
    portfolio_trading_record_df = portfolio_interday.tradingRecords

if portfolio_intraday_paths and portfolio_interday_paths:
    portfolio_df = pd.merge(portfolio_intraday.daily_report, portfolio_interday.daily_report, on=['Date'], how='outer')
    portfolio_df = portfolio_df.sort_values(by = ['Date']).reset_index(drop=True)
    portfolio_trading_record_df = pd.concat([portfolio_intraday.tradingRecords, portfolio_interday.tradingRecords]).sort_values(by = ['inDate']).reset_index(drop=True)

print('\r Completed intraday and interday portfolio preparation')
portfolio = Portfolio(portfolio_report = portfolio_df, portfolio_trading_record = portfolio_trading_record_df, portfolio_cols = portfolio_cols)



 Completed intraday and interday portfolio preparation


In [4]:

def get_occupy_ratio(hm):
    hm_line = hm.replace(0, np.nan).dropna().quantile(0.7)
    hm_adj = (hm_line - hm)
    hm_adj = hm_adj[hm_adj > 0]

    return  1 - hm_adj.sum() / (hm_line * len(hm_adj))

def merge_condition_hm_quantile(portfolio_df, pre_st_que, cur_st_que):
                
    #print(pre_st_que, cur_st_que)
    
    pre_profit, pre_hm = get_st_profit_hm(portfolio_df, pre_st_que)
    cur_profit, cur_hm = get_st_profit_hm(portfolio_df, cur_st_que)
    combined_profit, combined_hm = get_st_profit_hm(portfolio_df, pre_st_que + cur_st_que)
    
    pre_occupy_ratio = get_occupy_ratio(pre_hm)
    cur_occupy_ratio = get_occupy_ratio(cur_hm)
    combined_occupy_ratio = get_occupy_ratio(combined_hm)

    #print(pre_occupy_ratio, cur_occupy_ratio, combined_occupy_ratio)

    return combined_occupy_ratio > max(pre_occupy_ratio, cur_occupy_ratio) + 0.05
    

In [5]:

def get_preformance(profit_series, holding_money_series):
    
    return round(profit_series.sum() / 10000, 2), \
            round(get_MDD(profit_series) / 10000, 2), round(holding_money_series.cummax().iloc[-1] / 10000, 2), \
            round(holding_money_series[holding_money_series > 0].mean() / 10000, 2), \
        round((holding_money_series.max() - holding_money_series).sum() / (holding_money_series.max() * len(holding_money_series)), 2)

def portfolio_corr(portfolio_df, col1, col2):
    merge_df = pd.merge(portfolio_df[['Date', col1]], portfolio_df[['Date', col2]], on=['Date'], how='inner').dropna()
    merge_df = merge_df[(merge_df[col1] != 0) & (merge_df[col2] != 0)]
    return round(merge_df[col1].corr(merge_df[col2]), 2)



In [6]:

def get_st_profit_hm(portfolio_df, st_que):
    profit_list = [st + '_profit' for st in st_que]
    hm_list = [st + '_holding_money' for st in st_que]

    profit_series = portfolio_df[profit_list].sum(axis=1)
    holding_money_series = portfolio_df[hm_list].sum(axis=1)

    return profit_series, holding_money_series.dropna()

def get_st_info(portfolio_df, st_que):
    return get_preformance(*get_st_profit_hm(portfolio_df, st_que))

def merge_condition1(portfolio_df, pre_st_que, cur_st_que):

    max_holding_money_threshold = 1.1
    hm_ratio = 0.5
    mdd_threshold = 1.1
    #print(pre_st_que, cur_st_que)
    pre_np, pre_mdd, pre_max_hm, pre_mean_hm, pre_occupy_ratio = get_st_info(portfolio_df, pre_st_que)
    cur_np, cur_mdd, cur_max_hm, cur_mean_hm, cur_occupy_ratio = get_st_info(portfolio_df, cur_st_que)
    
    combined_np, combined_mdd, combined_max_hm, combined_mean_hm, combined_occupy_ratio = get_st_info(portfolio_df, pre_st_que + cur_st_que)
            
    return pre_max_hm / cur_max_hm > hm_ratio and pre_max_hm / cur_max_hm < 1 + hm_ratio \
    and combined_max_hm < max(pre_max_hm, cur_max_hm) * max_holding_money_threshold \
        and combined_mdd < max(pre_mdd, cur_mdd) * mdd_threshold


def merge_action(merge_condition, portfolio_report, portfolio_cols):
    portfolio_df = portfolio_report.copy()
    portfolio_deque = deque()
    for portfolio_col in portfolio_cols:
        portfolio_deque.append([portfolio_col])

    counter = 0
    while portfolio_deque:
        pre_st_que = portfolio_deque.popleft()
        if len(pre_st_que) > 1 or ( pre_st_que == [portfolio_cols[0]] and counter > 0 ) :
            portfolio_deque.append(pre_st_que)
            break
        
        portfolio_non_list = []
        while portfolio_deque:
            cur_st_que = portfolio_deque.popleft()
            #print(pre_st_que, cur_st_que)
            if merge_condition(portfolio_df, pre_st_que, cur_st_que):
                pre_st_que = pre_st_que + cur_st_que
            else:
                portfolio_non_list.append(cur_st_que)
                
        for portfolio_col in portfolio_non_list:
            portfolio_deque.append(portfolio_col)
        portfolio_deque.append(pre_st_que)
        counter += 1
        #print('portfolio_deque', portfolio_deque)
    
    return portfolio_deque


def merge_st_profit_hm(portfolio_df, st_que, merge_portfolio_cols):

    profit_list = [st + '_profit' for st in st_que]
    hm_list = [st + '_holding_money' for st in st_que]

    st_name = st_que[0]
    if len(st_que) > 1:
        for st in st_que[1:]:
            st_name += '|' + st 
    profit_name = st_name + '_profit'
    hm_name = st_name + '_holding_money'
    merge_portfolio_cols.append(st_name)

    profit_series = portfolio_df[profit_list].sum(axis=1)
    holding_money_series = portfolio_df[hm_list].sum(axis=1)

    portfolio_df[profit_name] = profit_series
    portfolio_df[hm_name] = holding_money_series



In [7]:

def portfolio_merge(merge_condition, portfolio_df, portfolio_trading_record_df, portfolio_cols):
    
    merge_portfolio_df = portfolio_df.copy()
    portfolio_deque = merge_action(merge_condition, merge_portfolio_df, portfolio_cols)
        
    merge_portfolio_cols = []
    for st_que in portfolio_deque:
        merge_st_profit_hm(merge_portfolio_df, st_que, merge_portfolio_cols)
        
    merge_portfolio_trading_record_df = portfolio_trading_record_df.copy()

    for cols in merge_portfolio_cols:
        for col in cols.split('|'):
            merge_portfolio_trading_record_df.loc[merge_portfolio_trading_record_df.strategy == col, 'strategy'] = cols
 
    return merge_portfolio_df, merge_portfolio_trading_record_df, merge_portfolio_cols


In [8]:

def get_portfolio_merge(portfolio_report, portfolio_cols):

    portfolio_df = portfolio_report.copy()
    
    strategy_merge_list = []
    
    for pre_index in range(len(portfolio_cols)-1):
        pre_profit_series = portfolio_df[f'{portfolio_cols[pre_index]}_profit']
        pre_holding_money_series = portfolio_df[f'{portfolio_cols[pre_index]}_holding_money']
        pre_np, pre_mdd, pre_max_hm, pre_mean_hm, pre_occupy_ratio = get_preformance(pre_profit_series, pre_holding_money_series.dropna())
       
        strategy_merge_list.append( [portfolio_cols[pre_index]] + [pre_np, pre_mdd, pre_max_hm, pre_mean_hm, pre_occupy_ratio, np.nan, np.nan])
        
        for cur_index in range(pre_index + 1, len(portfolio_cols)):
            cur_profit_series = portfolio_df[f'{portfolio_cols[cur_index]}_profit']
            cur_holding_money_series = portfolio_df[f'{portfolio_cols[cur_index]}_holding_money']
            cur_np, cur_mdd, cur_max_hm, cur_mean_hm, cur_occupy_ratio = get_preformance(cur_profit_series, cur_holding_money_series.dropna())

            profit_series = portfolio_df[[f'{portfolio_cols[pre_index]}_profit', f'{portfolio_cols[cur_index]}_profit']].sum(axis=1)
            holding_money_series = portfolio_df[[f'{portfolio_cols[pre_index]}_holding_money', f'{portfolio_cols[cur_index]}_holding_money']].sum(axis=1)
            combined_np, combined_mdd, combined_max_hm, combined_mean_hm, combined_occupy_ratio = get_preformance(profit_series, holding_money_series.dropna())
            
            
            profit_corr = portfolio_corr(portfolio_df, f'{portfolio_cols[pre_index]}_profit', f'{portfolio_cols[cur_index]}_profit')
            hm_corr = portfolio_corr(portfolio_df, f'{portfolio_cols[pre_index]}_holding_money', f'{portfolio_cols[cur_index]}_holding_money')

            strategy_merge_list.append([' + ' + portfolio_cols[cur_index]] + [combined_np, combined_mdd, combined_max_hm, combined_mean_hm, combined_occupy_ratio, profit_corr, hm_corr])
        strategy_merge_list.append([])
       
    strategy_merge_list.append([portfolio_cols[-1]] + [*get_preformance(portfolio_df[f'{portfolio_cols[-1]}_profit'], portfolio_df[f'{portfolio_cols[-1]}_holding_money'].dropna()), np.nan, np.nan])
    
    portfolio_merge_table = pd.DataFrame(strategy_merge_list, columns=['策略', '淨利(萬)', '區間最大回撤(萬)', '最大持倉金額(萬)', '平均持倉金額(萬)', '佔有比率', '損益相關性', '持倉金額相關性'])
    portfolio_merge_table.to_csv(report_path + '策略嵌合.csv', index = 0, encoding = 'big5')

    return portfolio_merge_table


In [9]:
get_portfolio_merge(portfolio_df, portfolio_cols)

,策略,淨利(萬),區間最大回撤(萬),最大持倉金額(萬),平均持倉金額(萬),佔有比率,損益相關性,持倉金額相關性
0,Win多2_玄哥(一日隔沖1雞酒1-1),376.21,44.98,1008.14,182.33,0.98,NaN,NaN
1,+ Win多2_玄哥(一日隔沖1雞酒1-2),1674.35,106.75,1519.52,305.32,0.95,0.16,0.06
2,+ Win多2_玄哥(一日隔沖1雞酒1-3),971.10,67.37,1976.04,215.85,0.98,0.10,-0.09
3,+ Win多2_玄哥(一日隔沖2雞酒2-1),1138.16,65.68,1377.84,428.97,0.69,0.23,0.05
4,+ Win多2_玄哥(一日隔沖2雞酒2-2),724.00,59.73,1513.74,236.74,0.97,0.05,-0.07
...,...,...,...,...,...,...,...,...
183,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184,老司機_1號,2473.10,307.64,4230.58,995.57,0.76,NaN,NaN
185,+ 處置股10日多,3932.21,403.36,6307.04,1573.73,0.75,0.47,0.41
186,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
total_asset = float(config['allocation']['total_asset'])
risk_factor = float(config['allocation']['risk_factor'])
strategy_risk_factor = float(config['allocation']['strategy_risk_factor'])
limited_holding_money = 10000000

In [11]:

def get_EMDD(period, profit_series):
    emdd = 0
    for index in range(period, len(profit_series)):
        emdd = max(emdd, get_MDD(profit_series[index-period:index]))
    return emdd

def get_portfolio_info_df(portfolio_df, portfolio_trading_record_df, portfolio_cols):
    portfolio_info_list = []

    for portfolio_col in portfolio_cols:
        holding_money_series = portfolio_df[f'{portfolio_col}_holding_money'].dropna()
        holding_money_series = holding_money_series[holding_money_series != 0]
        cur_portfolio_trading_record = portfolio_trading_record_df[portfolio_trading_record_df['strategy'] == portfolio_col]
        portfolio_info_list.append([portfolio_col, round(portfolio_df[f'{portfolio_col}_SLR'].iloc[-1], 2),
                                     round(get_MDD(portfolio_df[f'{portfolio_col}_profit']) /10000, 2),
                                   f'{round(holding_money_series.quantile(0.15) / 10000)} ~ {round(holding_money_series.quantile(0.85) / 10000)}',
                                   round(holding_money_series.max() / 10000),
        round((cur_portfolio_trading_record['profit'] * 10000 / ( cur_portfolio_trading_record['inPrice'] * cur_portfolio_trading_record['trading_number'] * 1000 * cur_portfolio_trading_record['holding_days'])).mean(), 2)])
        
    portfolio_info_df = pd.DataFrame(portfolio_info_list, columns=['策略', '槓桿率', '期望最大回撤(萬)', '持倉水位區間(萬)', '最大持倉金額(萬)', '單筆交易期望獲利(每萬/天)'])
    
    return portfolio_info_df


def compute_slr(portfolio_report, portfolio_trading_record, portfolio_cols, total_asset, risk_factor, strategy_risk_factor, risk_period, slr_limit = True):
    portfolio_df = portfolio_report.copy()
    portfolio_trading_record_df = portfolio_trading_record.copy()
    start_date = portfolio_df['Date'].iloc[0]

    ori_portfolio_df = portfolio_df[portfolio_df['Date'] >= backtesting_start_date].copy()
    ori_portfolio_trading_record_df = portfolio_trading_record_df[portfolio_trading_record_df['inDate'] >= backtesting_start_date].copy()

    portfolio_compare = []
    portfolio_compare.append(['Original', round(ori_portfolio_df['profit'].sum()/10000, 2), 
    round(get_MDD(ori_portfolio_df['profit'])/10000, 2),
    round(ori_portfolio_df['holding_money'].cummax().iloc[-1]/10000, 2) ])
    
    #ori_portfolio_info_df = get_portfolio_info_df(portfolio_df[portfolio_df['Date'] >= backtesting_start_date],
    # portfolio_trading_record_df[portfolio_trading_record_df['inDate'] >= backtesting_start_date], portfolio_cols)

    for portfolio_col in portfolio_cols:
            
        E_MDD = []
        mdd = 0
        for index in range(risk_period, len(portfolio_df)):
            mdd = max(mdd, get_MDD(portfolio_df[f'{portfolio_col}_profit'][index-risk_period:index]))
            E_MDD.append(mdd) 
            
        portfolio_df[f'{portfolio_col}_EMDD'] = [np.nan] * risk_period + E_MDD
        
    portfolio_df = portfolio_df.iloc[risk_period:].reset_index(drop=True)

    strategy_asset = (risk_factor * total_asset) / len(portfolio_cols)

    for portfolio_col in portfolio_cols:
        portfolio_df[f'{portfolio_col}_SLR'] =  (strategy_risk_factor / portfolio_df[f'{portfolio_col}_EMDD']) * strategy_asset
        if slr_limit:
            portfolio_df.loc[portfolio_df[f'{portfolio_col}_SLR'] > 1, f'{portfolio_col}_SLR'] = 1
        portfolio_df[f'{portfolio_col}_profit'] = portfolio_df[f'{portfolio_col}_profit'] * portfolio_df[f'{portfolio_col}_SLR'].shift(1)
        portfolio_df[f'{portfolio_col}_holding_money'] = portfolio_df[f'{portfolio_col}_holding_money'] * portfolio_df[f'{portfolio_col}_SLR'].shift(1)
        
    portfolio_trading_record_df = portfolio_trading_record_df[portfolio_trading_record_df['inDate'] >= portfolio_df['Date'].iloc[0]]
    portfolio_trading_record_df = portfolio_trading_record_df[portfolio_trading_record_df['strategy'].isin(portfolio_cols)]

    for index in portfolio_trading_record_df.index:
        strategy_name = portfolio_trading_record_df['strategy'].loc[index]
        cur_date = portfolio_trading_record_df['inDate'].loc[index]

        cur_SLR = portfolio_df[portfolio_df['Date'] <= cur_date][f'{strategy_name}_SLR'].iloc[-1]
        portfolio_trading_record_df.loc[index, 'trading_number'] *= cur_SLR
        portfolio_trading_record_df.loc[index, 'returns'] *= cur_SLR
        portfolio_trading_record_df.loc[index, 'profit'] *= cur_SLR

    portfolio_df = portfolio_df[portfolio_df['Date'] >= backtesting_start_date]
    portfolio_trading_record_df = portfolio_trading_record_df[portfolio_trading_record_df['inDate'] >= backtesting_start_date]

    portfolio_slr = Portfolio(name = f'portfolio_SLR_RF{risk_factor}_SRF{strategy_risk_factor}_Period{risk_period}',
                            portfolio_report = portfolio_df, portfolio_trading_record = portfolio_trading_record_df, portfolio_cols = portfolio_cols)
    portfolio_slr.get_plotly_html()

    portfolio_profit_cols = [col + '_profit' for col in portfolio_cols] 
    portfolio_holding_money_cols = [col + '_holding_money' for col in portfolio_cols] 
    portfolio_compare.append(['Adjusted', round(portfolio_df[portfolio_profit_cols].sum(axis=1).sum()/10000, 2),
    round(get_MDD(portfolio_df[portfolio_profit_cols].sum(axis=1))/10000, 2),
    round(portfolio_df[portfolio_holding_money_cols].sum(axis=1).cummax().iloc[-1]/10000, 2) ])
    portfolio_compare_df = pd.DataFrame(portfolio_compare, columns = ['投組狀態', ' 淨利(萬)', '最大回撤金額(萬)', '最大持倉金額(萬)'])
    
    portfolio_info_df = get_portfolio_info_df(portfolio_df, portfolio_trading_record_df, portfolio_cols)

    return portfolio_slr, portfolio_df, portfolio_trading_record_df, portfolio_compare_df, portfolio_info_df
    
    

In [16]:

portfolio_slr, portfolio_df_slr, portfolio_trading_record_df_slr, portfolio_compare_df, portfolio_info_df = compute_slr(portfolio_df, portfolio_trading_record_df, portfolio_cols, total_asset = total_asset, risk_factor = 0.15, strategy_risk_factor = 1, risk_period = 250)
 

In [17]:
portfolio_compare_df

,投組狀態,淨利(萬),最大回撤金額(萬),最大持倉金額(萬)
0,Original,11628.46,970.55,24803.39
1,Adjusted,4661.84,285.24,9102.05


In [18]:
portfolio_info_df

,策略,槓桿率,期望最大回撤(萬),持倉水位區間(萬),最大持倉金額(萬),單筆交易期望獲利(每萬/天)
0,Win多2_玄哥(一日隔沖1雞酒1-1),0.88,44.09,64 ~ 200,1008,31.35
1,Win多2_玄哥(一日隔沖1雞酒1-2),0.39,58.68,77 ~ 297,1143,53.99
2,Win多2_玄哥(一日隔沖1雞酒1-3),1.00,37.70,146 ~ 199,1976,31.26
3,Win多2_玄哥(一日隔沖2雞酒2-1),0.63,58.03,120 ~ 243,596,24.52
4,Win多2_玄哥(一日隔沖2雞酒2-2),1.00,39.04,151 ~ 375,1514,18.08
5,Win多3_玄哥(三日波段1即將創高),0.17,114.07,34 ~ 395,1944,-23.88
6,Win多3_玄哥(三日波段2投信突買),0.87,33.09,166 ~ 341,1265,46.02
7,Win多4_玄哥(五日波段1籌碼),0.33,19.68,32 ~ 183,490,25.83
8,Win多5_玄哥(十日波段1中大),0.58,42.42,101 ~ 397,954,-2.85
9,內部人10日多,0.37,46.23,70 ~ 215,762,-7.07


In [12]:

def get_occupy_ratio(hm):
    hm_line = hm.replace(0, np.nan).dropna().quantile(0.7)
    hm_adj = (hm_line - hm)
    hm_adj = hm_adj[hm_adj > 0]

    return  1 - hm_adj.sum() / (hm_line * len(hm_adj))

def merge_condition_hm_quantile(portfolio_df, pre_st_que, cur_st_que):
                
    #print(pre_st_que, cur_st_que)
    
    pre_profit, pre_hm = get_st_profit_hm(portfolio_df, pre_st_que)
    cur_profit, cur_hm = get_st_profit_hm(portfolio_df, cur_st_que)
    combined_profit, combined_hm = get_st_profit_hm(portfolio_df, pre_st_que + cur_st_que)
    
    pre_MDD, cur_MDD, combined_MDD = get_MDD(pre_profit), get_MDD(cur_profit), get_MDD(combined_profit)
    pre_MDD_ratio, cur_MDD_ratio, combined_MDD_ratio = pre_MDD / pre_hm.quantile(0.9), cur_MDD / cur_hm.quantile(0.9), combined_MDD / combined_hm.quantile(0.9)

    pre_occupy_ratio = get_occupy_ratio(pre_hm)
    cur_occupy_ratio = get_occupy_ratio(cur_hm)
    combined_occupy_ratio = get_occupy_ratio(combined_hm)
    
    #print(pre_occupy_ratio, cur_occupy_ratio, combined_occupy_ratio)

    return combined_occupy_ratio > max(pre_occupy_ratio, cur_occupy_ratio) + 0.05 and combined_MDD < pre_MDD + cur_MDD and \
         (combined_MDD_ratio < pre_MDD_ratio or combined_MDD_ratio < cur_MDD_ratio)
    

In [13]:

def judge_hm_usage(portfolio_df, pre_st_que, cur_st_que):
    hm_df = pd.DataFrame()
    hm_df['date'] = portfolio_df['Date']

    pre_hm_list = [st + '_holding_money' for st in pre_st_que]
    cur_hm_list = [st + '_holding_money' for st in cur_st_que]

    hm_df['pre_hm'] = portfolio_df[pre_hm_list].sum(axis=1)
    hm_df['cur_hm'] = portfolio_df[cur_hm_list].sum(axis=1)

        
    hm_usage_count = len(hm_df[(hm_df['pre_hm'] != 0) & (hm_df['cur_hm'] != 0)])

    hm_usage_ratio = hm_usage_count / len(hm_df)

    return hm_usage_ratio < 0.01


In [14]:

def merge_condition_synthesized(portfolio_df, pre_st_que, cur_st_que):

    pre_np, pre_sharpe_ratio, pre_sortino_ratio, pre_mdd_money, pre_mdd_ratio, pre_max_hm, pre_occupy_ratio, pre_vol, pre_beta, pre_alpha = evaluate_portfolio(portfolio_df, pre_st_que)
    cur_np, cur_sharpe_ratio, cur_sortino_ratio, cur_mdd_money, cur_mdd_ratio, cur_max_hm, cur_occupy_ratio, cur_vol, cur_beta, cur_alpha = evaluate_portfolio(portfolio_df, cur_st_que)
    combined_np, combined_sharpe_ratio, combined_sortino_ratio, combined_mdd_money, combined_mdd_ratio, combined_max_hm, combined_occupy_ratio, combined_vol, combined_beta, combined_alpha = evaluate_portfolio(portfolio_df, pre_st_que + cur_st_que)
    
    return combined_mdd_ratio < min(pre_mdd_ratio, cur_mdd_ratio) and (combined_occupy_ratio > max(pre_occupy_ratio, cur_occupy_ratio))


In [27]:

def get_portfolio_df_slr_2stage(portfolio_df, portfolio_trading_record_df, portfolio_df_slr):

    portfolio_df_slr_2stage = portfolio_df.copy().reset_index(drop=True)
    portfolio_trading_record_df_slr_2stage = portfolio_trading_record_df

    for i in range(len(portfolio_df_slr_2stage)):
        Date = portfolio_df_slr_2stage['Date'].iloc[i]
        for portfolio_col in portfolio_cols:
            if not portfolio_df_slr.loc[portfolio_df_slr['Date'] == Date, portfolio_col + '_SLR'].empty and portfolio_df_slr_2stage.loc[i, portfolio_col + '_profit']:
                portfolio_df_slr_2stage.loc[i, portfolio_col + '_profit'] *= portfolio_df_slr.loc[portfolio_df_slr['Date'] == Date, portfolio_col + '_SLR'].values[0]
                portfolio_df_slr_2stage.loc[i, portfolio_col + '_holding_money'] *= portfolio_df_slr.loc[portfolio_df_slr['Date'] == Date, portfolio_col + '_SLR'].values[0]

    portfolio_trading_record_df = portfolio_trading_record_df[portfolio_trading_record_df['strategy'].isin(portfolio_cols)]

    for index in portfolio_trading_record_df_slr_2stage.index:
        strategy_name = portfolio_trading_record_df_slr_2stage['strategy'].loc[index]
        cur_date = portfolio_trading_record_df_slr_2stage['inDate'].loc[index]
        
        if not portfolio_df_slr[portfolio_df_slr['Date'] <= cur_date][f'{strategy_name}_SLR'].empty:
            cur_SLR = portfolio_df_slr[portfolio_df_slr['Date'] <= cur_date][f'{strategy_name}_SLR'].iloc[-1]
            portfolio_trading_record_df_slr_2stage.loc[index, 'trading_number'] *= cur_SLR
            portfolio_trading_record_df_slr_2stage.loc[index, 'returns'] *= cur_SLR
            portfolio_trading_record_df_slr_2stage.loc[index, 'profit'] *= cur_SLR

    return portfolio_df_slr_2stage, portfolio_trading_record_df_slr_2stage

portfolio_df_slr_2stage, portfolio_trading_record_df_slr_2stage = get_portfolio_df_slr_2stage(portfolio_df, portfolio_trading_record_df, portfolio_df_slr)



In [26]:
#merge_portfolio_df, merge_portfolio_trading_record_df, merge_portfolio_cols = portfolio_merge(merge_condition_synthesized, portfolio_df_slr_2stage, portfolio_trading_record_df_slr_2stage, portfolio_cols)

In [28]:
merge_portfolio_df, merge_portfolio_trading_record_df, merge_portfolio_cols = portfolio_merge(merge_condition_synthesized, portfolio_df, portfolio_trading_record_df, portfolio_cols)

In [15]:
merge_portfolio_df, merge_portfolio_trading_record_df, merge_portfolio_cols = portfolio_merge(judge_hm_usage, portfolio_df, portfolio_trading_record_df, portfolio_cols)

In [16]:

merge_portfolio_slr, merge_portfolio_df_slr, merge_portfolio_trading_record_df_slr, merge_portfolio_compare_df, merge_portfolio_info_df = compute_slr(merge_portfolio_df, merge_portfolio_trading_record_df, merge_portfolio_cols, total_asset = total_asset, risk_factor = 0.14, strategy_risk_factor = 1, risk_period = 250, slr_limit = True)


In [17]:
merge_portfolio_compare_df

,投組狀態,淨利(萬),最大回撤金額(萬),最大持倉金額(萬)
0,Original,11486.57,970.55,30115.07
1,Adjusted,4806.23,297.81,11499.61


In [18]:
merge_portfolio_info_df

,策略,槓桿率,期望最大回撤(萬),持倉水位區間(萬),最大持倉金額(萬),單筆交易期望獲利(每萬/天)
0,Win多2_玄哥(一日隔沖1雞酒1-2),0.41,61.21,80 ~ 310,1192,53.99
1,Win多2_玄哥(一日隔沖1雞酒1-3),1.00,37.70,146 ~ 199,1976,31.26
2,Win多2_玄哥(一日隔沖2雞酒2-1),0.65,58.92,241 ~ 419,965,24.52
3,Win多2_玄哥(一日隔沖2雞酒2-2),1.00,39.04,151 ~ 375,1514,18.08
4,Win多3_玄哥(三日波段1即將創高),0.18,118.99,35 ~ 395,2028,-23.88
5,Win多3_玄哥(三日波段2投信突買),0.90,33.22,173 ~ 356,1320,46.02
6,Win多4_玄哥(五日波段1籌碼),0.35,20.53,643 ~ 779,1155,25.83
7,Win多5_玄哥(十日波段1中大),0.60,44.24,105 ~ 414,995,-2.85
8,內部人10日多,0.38,48.23,73 ~ 224,795,-7.07
9,法說會10日多,0.34,40.85,67 ~ 369,660,24.89


In [44]:
portfolio_info_df

,策略,槓桿率,期望最大回撤(萬),持倉水位區間(萬),最大持倉金額(萬),單筆交易期望獲利(每萬/天)
0,Win多2_玄哥(一日隔沖1雞酒1-1),1.00,27.39,85 ~ 101,492,5.92
1,Win多2_玄哥(一日隔沖1雞酒1-2),0.56,46.52,55 ~ 195,788,74.37
2,Win多2_玄哥(一日隔沖1雞酒1-3),1.00,29.84,89 ~ 99,1048,36.84
3,Win多2_玄哥(一日隔沖2雞酒2-1),1.00,31.90,86 ~ 167,295,37.45
4,Win多2_玄哥(一日隔沖2雞酒2-2),1.00,22.51,65 ~ 187,700,33.14
5,Win多3_玄哥(三日波段1即將創高),0.28,88.29,28 ~ 197,1279,17.45
6,Win多3_玄哥(三日波段2投信突買),1.00,32.49,93 ~ 100,760,60.66
7,Win多4_玄哥(五日波段1籌碼),0.51,29.34,50 ~ 243,521,17.69
8,Win多5_玄哥(十日波段1中大),0.79,40.30,74 ~ 273,577,2.40
9,內部人10日多,0.67,32.43,63 ~ 187,576,0.14


In [19]:

def get_dd_mdd_series(profit_series):
    return  profit_series.cumsum() - profit_series.cumsum().cummax(), (profit_series.cumsum() - profit_series.cumsum().cummax()).cummin()

def plot_show(fig, title_text):
    fig.update_layout(bargap=0)
    fig.update_layout(barmode='stack')
    fig.update_layout(
                        height = 500, 
                        width = 1200, #1700
                        title = {'text': f'<b>{title_text}<b>', 'x': 0.5, 'font': {'size': 30}}
                        )
    fig.show()

def get_profit_hm_series(portfolio_df, portfolio_cols):
    portfolio_profit_cols = [portfolio_col + '_profit' for portfolio_col in portfolio_cols]
    portfolio_hm_cols = [portfolio_col + '_holding_money' for portfolio_col in portfolio_cols]
    
    profit_series = portfolio_df[portfolio_profit_cols].sum(axis=1)
    hm_series = portfolio_df[portfolio_hm_cols].sum(axis=1)
    return profit_series, hm_series

def add_plot(fig_profit, fig_hm, fig_dd, fig_mdd, portfolio_df, portfolio_cols, portfolio_name):
    
    profit_series, hm_series = get_profit_hm_series(portfolio_df, portfolio_cols)
    dd, mdd = get_dd_mdd_series(profit_series)

    fig_profit.add_trace(go.Scatter(x= portfolio_df['Date'], y=profit_series.cumsum(), name=portfolio_name))
    fig_hm.add_trace(go.Scatter(x= portfolio_df['Date'], y=hm_series, name=portfolio_name))
    fig_dd.add_trace(go.Scatter(x= portfolio_df['Date'], y=dd, name=portfolio_name))
    fig_mdd.add_trace(go.Scatter(x= portfolio_df['Date'], y=mdd, name=portfolio_name))
    

def plot_portfolio_compare(portfolio_base, portfolio_compare, show=True):
    fig_profit = go.Figure()
    fig_hm = go.Figure()
    fig_dd = go.Figure()
    fig_mdd = go.Figure()

    portfolio_info_list = []

    portfolio_df_base, portfolio_cols_base, portfolio_name_base = portfolio_base
    portfolio_df_base = portfolio_df_base[portfolio_df_base['Date'] >= backtesting_start_date]

    profit_base, hm_base = get_profit_hm_series(portfolio_df_base, portfolio_cols_base)
    dd_base, mdd_base = get_dd_mdd_series(profit_base)

    portfolio_info_list.append( [portfolio_name_base, *evaluate_portfolio(portfolio_df_base, portfolio_cols_base)])

    add_plot(fig_profit, fig_hm, fig_dd, fig_mdd, portfolio_df_base, portfolio_cols_base, portfolio_name_base)

    adjusted_info_list = []

    for portfolio_df, portfolio_cols, portfolio_name in portfolio_compare:
    
        portfolio_df = portfolio_df[portfolio_df['Date'] >= backtesting_start_date]
        profit, hm = get_profit_hm_series(portfolio_df, portfolio_cols)
        dd, mdd = get_dd_mdd_series(profit)
        
        adjusted_info_list.append([portfolio_name, round(hm.sum()/hm_base.sum(), 2), round(profit.sum()/profit_base.sum(), 2), 
        round(dd.sum()/ dd_base.sum(), 2), round(mdd.min()/mdd_base.min(), 2)])
        portfolio_info_list.append([portfolio_name, *evaluate_portfolio(portfolio_df, portfolio_cols)])

        add_plot(fig_profit, fig_hm, fig_dd, fig_mdd, portfolio_df, portfolio_cols, portfolio_name)
        #add_plot(fig_profit, fig_hm, fig_dd, fig_mdd, merge_portfolio_df, merge_portfolio_cols, 'Adjusted')

    if show:
        plot_show(fig_profit, '投組損益')
        plot_show(fig_hm, '投組持倉水位')
        plot_show(fig_dd, '投組DD')
        plot_show(fig_mdd, '投組MDD')
    
    return pd.DataFrame(adjusted_info_list, columns=['方法', '持倉佔比變動', '獲利變動', 'DD變動', 'MDD變動']), \
            pd.DataFrame(portfolio_info_list, columns=['方法', '淨利(萬)', 'Sharpe Ratio', 'Sortino Ratio', 'MDD(萬)', "MDD(%)", '最大持倉金額(萬)', "佔有比率(%)", '年化波動率(%)', 'Beta', 'Alpha'])
    #return fig_pos



In [75]:


for merge_portfolio_col in merge_portfolio_cols:
    
    merge_portfolio_cols_list = merge_portfolio_col.split('|')
    print(merge_portfolio_cols_list)
    original = (portfolio_df, merge_portfolio_cols_list, 'Original')
    adjusted = [(merge_portfolio_df_slr, [merge_portfolio_col], 'Adjusted')]

    adjusted_info, portfolio_info = plot_portfolio_compare(original, adjusted)

    print(adjusted_info)
    print(portfolio_info)
    


['Win多3_玄哥(三日波段2投信突買)', '注意股10日多']


         方法  持倉佔比變動  獲利變動  DD變動  MDD變動
0  Adjusted    0.89  0.88  0.89    1.0
         方法   淨利(萬)  Sharpe Ratio  Sortino Ratio  最大回撤金額(萬)  MDD(%)  \
0  Original  737.58          1.62           1.72     117.11   11.73   
1  Adjusted  650.07          1.57           1.66     117.11   12.58   

   最大持倉金額(萬)  佔有比率(%)  年化波動率(%)  Beta     Alpha  
0    1981.74    30.77     18.87  0.17  0.000900  
1    1718.61    31.45     18.34  0.18  0.000857  
['內部人10日多', 'Win多3_玄哥(三日波段1即將創高)', '營收首次創新高20日多', '用月營收預估的低本益比股20日多', '老司機_1號', '處置股10日多']


         方法  持倉佔比變動  獲利變動  DD變動  MDD變動
0  Adjusted    0.35  0.35  0.35   0.38
         方法    淨利(萬)  Sharpe Ratio  Sortino Ratio  最大回撤金額(萬)  MDD(%)  \
0  Original  2418.15          2.00           2.27     308.61    9.71   
1  Adjusted   836.99          1.84           2.02     117.31    9.54   

   最大持倉金額(萬)  佔有比率(%)  年化波動率(%)  Beta     Alpha  
0    8305.99    45.76     15.74  0.21  0.000896  
1    3256.00    44.03     15.22  0.20  0.000820  
['Win多2_玄哥(一日隔沖1雞酒1-1)', 'Win多2_玄哥(一日隔沖1雞酒1-2)', 'Win多2_玄哥(一日隔沖1雞酒1-3)', 'Win多2_玄哥(一日隔沖2雞酒2-1)', 'Win多2_玄哥(一日隔沖2雞酒2-2)', 'Win多4_玄哥(五日波段1籌碼)', 'Win多5_玄哥(十日波段1中大)', '法說會10日多', '籌碼雷達5日多']


         方法  持倉佔比變動  獲利變動  DD變動  MDD變動
0  Adjusted     1.0   1.0   1.0    1.0
         方法    淨利(萬)  Sharpe Ratio  Sortino Ratio  最大回撤金額(萬)  MDD(%)  \
0  Original  2749.48          3.83           6.57      95.16    5.37   
1  Adjusted  2749.48          3.83           6.57      95.16    5.37   

   最大持倉金額(萬)  佔有比率(%)  年化波動率(%)  Beta    Alpha  
0    4049.16    54.33     17.03  0.19  0.00153  
1    4049.16    54.33     17.03  0.19  0.00153  


In [84]:

portfolio_base = (portfolio_df, portfolio_cols, '原始')
portfolio_compare = [(portfolio_df_slr, portfolio_cols, '槓桿率調整'),
                    (merge_portfolio_df_slr, merge_portfolio_cols, '策略嵌合')]

adjusted_info, portfolio_info = plot_portfolio_compare(portfolio_base, portfolio_compare, show=True)


In [85]:
adjusted_info

,方法,持倉佔比變動,獲利變動,DD變動,MDD變動
0,槓桿率調整,0.47,0.54,0.40,0.45
1,策略嵌合,0.67,0.72,0.58,0.57


In [86]:
portfolio_info

,方法,淨利(萬),Sharpe Ratio,Sortino Ratio,MDD(萬),MDD(%),最大持倉金額(萬),佔有比率(%),年化波動率(%),Beta,Alpha
0,原始,5905.21,2.90,3.61,450.23,8.09,13157.33,52.90,15.28,0.21,0.001158
1,槓桿率調整,3170.83,3.31,4.62,200.95,7.66,6716.04,52.95,15.28,0.19,0.001278
2,策略嵌合,4236.54,3.17,4.28,256.77,7.89,8107.34,55.02,17.18,0.22,0.001343


In [77]:
merge_portfolio_info_df

,策略,槓桿率,期望最大回撤(萬),持倉水位區間(萬),最大持倉金額(萬),單筆交易期望獲利(每萬/天)
0,Win多3_玄哥(三日波段2投信突買)|注意股10日多,0.85,117.11,84 ~ 566,1719,36.79
1,內部人10日多|Win多3_玄哥(三日波段1即將創高)|營收首次創新高20日多|用月營收預估...,0.32,117.31,68 ~ 608,3256,3.29
2,Win多2_玄哥(一日隔沖1雞酒1-1)|Win多2_玄哥(一日隔沖1雞酒1-2)|Win多...,1.00,95.16,286 ~ 1357,4049,17.75


In [387]:
plot_holding_money_bar(portfolio_df_slr, merge_portfolio_cols)

In [20]:


from tsdb_client import TSDBClient
from utils.Config import EnvConfig

cli = TSDBClient(
    host=EnvConfig.DB_HOST,
    port=EnvConfig.DB_PORT,
    user=EnvConfig.DB_USER,
    password=EnvConfig.DB_PASSWORD,
    db=EnvConfig.DB_DATABASE,
)


def upload_portfolio_info(portfolio_info_df):

    for i in range(len(portfolio_info_df)):
        strategy_names = portfolio_info_df['策略'].iloc[i].split('|')
        
        for strategy_name in strategy_names:

            lr = min(1, portfolio_info_df['槓桿率'].iloc[i]) 
            expected_mdd = portfolio_info_df['期望最大回撤(萬)'].iloc[i]
            expected_daily_return = portfolio_info_df['單筆交易期望獲利(每萬/天)'].iloc[i]	
                
            cli.execute_query(f'''
            UPDATE dealer.strategy SET leverage_ratio = {lr}, expected_mdd = {expected_mdd}, expected_daily_return = {expected_daily_return} WHERE name = '{strategy_name}'
            ''')


In [21]:
upload_portfolio_info(merge_portfolio_info_df)

In [138]:
strategy_info = {}

for i in range(len(portfolio_info_df)):
    strategy_name = portfolio_info_df.iloc[i]['策略']
    strategy_info[strategy_name] = {}
    strategy_info[strategy_name]['Leverage_Ratio'] = portfolio_info_df.iloc[i]['槓桿率']
    strategy_info[strategy_name]['Expected_MDD'] = portfolio_info_df.iloc[i]['期望最大回撤(萬)']
    strategy_info[strategy_name]['Expected_Holding_Money'] = portfolio_info_df.iloc[i]['持倉水位區間(萬)']
    strategy_info[strategy_name]['Expected_Daily_Return'] = portfolio_info_df.iloc[i]['單筆交易期望獲利(每萬/天)']


In [139]:
import json
with open('./log/strategy_info.json', "w") as f:
    json.dump(strategy_info, f)

In [27]:
with open('./log/strategy_info.json', "r") as f:
    strategy_info = json.load(f)

In [291]:


def get_portfolio_holding_money_fig(portfolio_df, portfolio_cols):
    
    fig_pos = go.Figure()
    for portfolio_col in portfolio_cols:
        fig_pos.add_trace(go.Scatter(x=portfolio_df['Date'], y=portfolio_df[portfolio_col + '_holding_money'], name=portfolio_col))
    fig_pos.add_trace(go.Scatter(x =portfolio_df['Date'], y = portfolio_df[[portfolio_col + '_holding_money' for portfolio_col in portfolio_cols]].sum(axis=1), name = 'Portfolio', line=dict(color='royalblue', width=4, dash='dash')))

    fig_pos.update_layout(bargap=0)
    fig_pos.update_layout(barmode='stack')
    fig_pos.update_layout(
                        height = 700, 
                        width = 1700, #1700
                        title = {'text': '<b>策略(投組)持倉水位圖<b>', 'x': 0.5, 'font': {'size': 30}}
                        )
    #fig_pos.show()
    return fig_pos
    

def plot_holding_money_box(portfolio_df, portfolio_cols):
    fig_pos = go.Figure()
    for portfolio_col in portfolio_cols:
        holding_money_series = portfolio_df[f'{portfolio_col}_holding_money'].dropna()
        holding_money_series = holding_money_series[holding_money_series != 0]
        fig_pos.add_trace(go.Box( y=holding_money_series, name=portfolio_col))
    holding_money_series = portfolio_df[[portfolio_col + '_holding_money' for portfolio_col in portfolio_cols]].sum(axis=1).dropna()
    holding_money_series = holding_money_series[holding_money_series != 0]
    fig_pos.add_trace(go.Box(y = holding_money_series, name = 'Portfolio'))

    fig_pos.update_layout(bargap=0)
    fig_pos.update_layout(barmode='stack')
    fig_pos.update_layout(
                        height = 700, 
                        width = 1700, #1700
                        title = {'text': '<b>策略(投組)持倉水位圖<b>', 'x': 0.5, 'font': {'size': 30}}
                        )
    return fig_pos


def plot_holding_money_bar(portfolio_df, portfolio_cols):
    fig_pos = go.Figure()
    for portfolio_col in portfolio_cols:
        fig_pos.add_trace(go.Bar(x= portfolio_df['Date'], y=portfolio_df[portfolio_col + '_holding_money'], name=portfolio_col))
    #fig_pos.add_trace(go.Bar(y = portfolio_df[[portfolio_col + '_holding_money' for portfolio_col in portfolio_cols]].sum(axis=1), name = 'Portfolio'))

    fig_pos.update_layout(bargap=0)
    fig_pos.update_layout(barmode='stack')
    fig_pos.update_layout(barmode='relative', title_text='Relative Barmode')
    fig_pos.update_layout(
                        height = 700, 
                        width = 1700, #1700
                        title = {'text': '<b>策略(投組)持倉水位圖<b>', 'x': 0.5, 'font': {'size': 30}}
                        )
    return fig_pos

def plot_holding_money_line(portfolio_df, portfolio_cols):
    fig_pos = go.Figure()
    for portfolio_col in portfolio_cols:
        fig_pos.add_trace(go.Scatter(x= portfolio_df['Date'], y=portfolio_df[portfolio_col + '_holding_money'], name=portfolio_col))
    #fig_pos.add_trace(go.Bar(y = portfolio_df[[portfolio_col + '_holding_money' for portfolio_col in portfolio_cols]].sum(axis=1), name = 'Portfolio'))

    fig_pos.update_layout(bargap=0)
    fig_pos.update_layout(barmode='stack')
    fig_pos.update_layout(
                        height = 700, 
                        width = 1700, #1700
                        title = {'text': '<b>策略(投組)持倉水位圖<b>', 'x': 0.5, 'font': {'size': 30}}
                        )
    return fig_pos


In [30]:
def get_portfolio_corr(portfolio_df, portfolio_cols, cols):
    
    holding_money_corr = portfolio_df[cols].corr()
    holding_money_corr.index = portfolio_cols 
    holding_money_corr.columns = portfolio_cols 
    holding_money_corr = sns.heatmap(holding_money_corr, annot=True, vmax=1, square=True, fmt=".2f", cmap='coolwarm')

    #return holding_money_corr
    
#get_portfolio_corr(portfolio_df, portfolio_cols, [col + '_holding_money' for col in portfolio_cols] )


In [292]:
plot_holding_money_line(portfolio_df_slr, portfolio_cols)

In [293]:
plot_holding_money_bar(portfolio_df_slr, merge_portfolio_cols)

In [35]:
plot_holding_money_box(portfolio_df_slr, portfolio_cols)